## <center>                            Microsoft Volatility Analysis Project 

####  <center>                  presented by Brian Zamkotowicz for Springboard Capstone Project

In the following project I intend to do a time series analysis of volatility in Microsoft stock.
Specifically, I will see if the implied volatility of at the money option can be predicted first by looking at the implied volatilities over time (time-series analysis), and then by also including other features into the analysis.  From a business perspective these predictions, if deemed successful, could be used to implement option trading strategies, to produce profits, based upon those predictions.

###  <center> Part 1 - Data Wrangling
The purpose of the following notebook will be to clean and prepare data for an analysis of volatility in Microsoft stock.  The data on Microsoft stock and options is available for free on quandl.com and is available to anyone who downloads an API key.  The data on the stock itself is provided separately from the option data and includes items such as high, low, close, and adjusted (taking into account splits) stock price.  The option data provides insight into both historical volatility of the stock as well as the implied volatility of options of different maturity lengths.  In order to use this data it will have to be cleaned by eliminating entries with NaN's and organized into appropriate categories.  Additionally, the stock and option data needs to be combined into a useful data frame.

In [2]:
import plotly


In [3]:
init_notebook_mode(connected=True)

NameError: name 'init_notebook_mode' is not defined

In [4]:
from pyramid.arima import auto_arima
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


C:\Conda\lib\site-packages\pyramid\__init__.py:68: UserWarning:


    The 'pyramid' package will be migrating to a new namespace beginning in 
    version 1.0.0: 'pmdarima'. This is due to a package name collision with the
    Pyramid web framework. For more information, see Issue #34:
    
        https://github.com/tgsmith61591/pyramid/issues/34
        
    The package will subsequently be installable via the name 'pmdarima'; the
    only functional change to the user will be the import name. All imports
    from 'pyramid' will change to 'pmdarima'.
    



In [5]:
import quandl
import plotly.plotly as ply
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from datetime import datetime

In [6]:
#make a call to quandl api for microsoft end of day prices as well as several other variables

csv = "https://www.quandl.com/api/v3/datasets/WIKI/MSFT/data.csv"
MSFT = pd.read_csv(csv)
MSFT.head()


,Date,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
0,2018-03-27,94.940,95.139,88.51,89.47,53704562.0,0.0,1.0,94.940,95.139,88.51,89.47,53704562.0
1,2018-03-26,90.610,94.000,90.40,93.78,55031149.0,0.0,1.0,90.610,94.000,90.40,93.78,55031149.0
2,2018-03-23,89.500,90.460,87.08,87.18,42159397.0,0.0,1.0,89.500,90.460,87.08,87.18,42159397.0
3,2018-03-22,91.265,91.750,89.66,89.79,37578166.0,0.0,1.0,91.265,91.750,89.66,89.79,37578166.0
4,2018-03-21,92.930,94.050,92.21,92.48,23753263.0,0.0,1.0,92.930,94.050,92.21,92.48,23753263.0


In [7]:
MSFT['Date'] = pd.to_datetime(MSFT.Date)
MSFT.dtypes


Date           datetime64[ns]
Open                  float64
High                  float64
Low                   float64
Close                 float64
Volume                float64
Ex-Dividend           float64
Split Ratio           float64
Adj. Open             float64
Adj. High             float64
Adj. Low              float64
Adj. Close            float64
Adj. Volume           float64
dtype: object

In [8]:
MSFT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8076 entries, 0 to 8075
Data columns (total 13 columns):
Date           8076 non-null datetime64[ns]
Open           8076 non-null float64
High           8076 non-null float64
Low            8076 non-null float64
Close          8076 non-null float64
Volume         8076 non-null float64
Ex-Dividend    8076 non-null float64
Split Ratio    8076 non-null float64
Adj. Open      8076 non-null float64
Adj. High      8076 non-null float64
Adj. Low       8076 non-null float64
Adj. Close     8076 non-null float64
Adj. Volume    8076 non-null float64
dtypes: datetime64[ns](1), float64(12)
memory usage: 820.3 KB


The data seems to have the same number of entries in each category.  I will make sure there are no NaN entires as well.


In [9]:
MSFT.count()

Date           8076
Open           8076
High           8076
Low            8076
Close          8076
Volume         8076
Ex-Dividend    8076
Split Ratio    8076
Adj. Open      8076
Adj. High      8076
Adj. Low       8076
Adj. Close     8076
Adj. Volume    8076
dtype: int64

The number of non-NaN values seems to match the number of values, so this data can be used going forward.

In [10]:
# call quand API for  microsoft option volatilities

VOLcsv = "https://www.quandl.com/api/v3/datasets/VOL/MSFT.csv?api_key=zJY23nxcWpazQUQdc-AB"
vol = pd.read_csv(VOLcsv)
vol['Date'] = pd.to_datetime(vol.Date)
vol.head()

,Date,Hv10,Hv20,Hv30,Hv60,Hv90,Hv120,Hv150,Hv180,Phv10,...,IvMean360,IvMeanSkew360,IvCall720,IvPut720,IvMean720,IvMeanSkew720,IvCall1080,IvPut1080,IvMean1080,IvMeanSkew1080
0,2019-04-23,0.0951,0.0937,0.0938,0.1363,0.1786,0.2514,0.2730,0.2990,0.1160,...,0.2088,0.0164,0.2108,0.2563,0.2336,0.0214,0.2108,0.2563,0.2336,0.0214
1,2019-04-22,0.0819,0.0825,0.1337,0.1367,0.1779,0.2540,0.2781,0.2987,0.1080,...,0.2141,0.0187,0.2029,0.2570,0.2300,0.0196,0.2029,0.2570,0.2300,0.0196
2,2019-04-18,0.0839,0.0867,0.1528,0.1437,0.1831,0.2542,0.2784,0.2987,0.1026,...,0.2103,0.0216,0.2105,0.2394,0.2250,0.0030,0.2105,0.2394,0.2250,0.0030
3,2019-04-17,0.0809,0.0786,0.1480,0.1431,0.1836,0.2539,0.2796,0.2997,0.0843,...,0.2188,0.0183,0.2033,0.2449,0.2241,0.0216,0.2033,0.2449,0.2241,0.0216
4,2019-04-16,0.0691,0.0749,0.1461,0.1427,0.1835,0.2597,0.2805,0.2996,0.0855,...,0.2195,0.0180,0.1983,0.2500,0.2242,0.0135,0.1983,0.2500,0.2242,0.0135


In [11]:
vol.shape

(4294, 65)

In [12]:
MSFT.shape

(8076, 13)

In [13]:
MSFT.tail()

,Date,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
8071,1986-03-19,28.75,29.00,28.0,28.25,166300.0,0.0,1.0,0.066454,0.067031,0.064720,0.065298,4.789440e+07
8072,1986-03-18,29.50,29.75,28.5,28.75,235300.0,0.0,1.0,0.068187,0.068765,0.065876,0.066454,6.776640e+07
8073,1986-03-17,29.00,29.75,29.0,29.50,462400.0,0.0,1.0,0.067031,0.068765,0.067031,0.068187,1.331712e+08
8074,1986-03-14,28.00,29.50,28.0,29.00,1070000.0,0.0,1.0,0.064720,0.068187,0.064720,0.067031,3.081600e+08
8075,1986-03-13,25.50,29.25,25.5,28.00,3582600.0,0.0,1.0,0.058941,0.067609,0.058941,0.064720,1.031789e+09


In [14]:
vol.tail()

,Date,Hv10,Hv20,Hv30,Hv60,Hv90,Hv120,Hv150,Hv180,Phv10,...,IvMean360,IvMeanSkew360,IvCall720,IvPut720,IvMean720,IvMeanSkew720,IvCall1080,IvPut1080,IvMean1080,IvMeanSkew1080
4289,2002-03-21,0.3108,0.3058,0.3327,NaN,NaN,NaN,NaN,NaN,0.2425,...,0.3466,0.0336,0.3641,0.3653,0.3647,0.0257,0.3641,0.3653,0.3647,0.0257
4290,2002-03-20,0.2944,0.3656,0.3297,NaN,NaN,NaN,NaN,NaN,0.2284,...,0.3521,0.0301,0.3707,0.3670,0.3688,0.0241,0.3707,0.3670,0.3688,0.0241
4291,2002-03-19,0.2463,0.3252,0.3147,NaN,NaN,NaN,NaN,NaN,0.2281,...,0.3468,0.0276,0.3722,0.3656,0.3689,0.0230,0.3722,0.3656,0.3689,0.0230
4292,2002-03-18,0.2835,0.3265,0.3267,NaN,NaN,NaN,NaN,NaN,0.2279,...,0.3507,0.0306,0.3740,0.3643,0.3691,0.0240,0.3740,0.3643,0.3691,0.0240
4293,2002-03-15,0.2938,0.3288,0.3261,NaN,NaN,NaN,NaN,NaN,0.2335,...,0.3499,0.0296,0.3729,0.3674,0.3702,0.0239,0.3729,0.3674,0.3702,0.0239


Microsoft price data goes back to 1985, while option data only goes back to 2002.  This is fine as I doubt any useful information will be gleaned from 1985 to 2002 data.  Also the volatility of longer dated options seems to contain a number of NaN's to be dealt with later.

In [15]:
vol = vol.drop(columns =['Hv60', 'Hv90', 'Hv120', 'Hv150', 'Hv180', 'Phv60', 'Phv90', 'Phv120', 'Phv150', 'Phv180'])
vol.tail()

,Date,Hv10,Hv20,Hv30,Phv10,Phv20,Phv30,IvCall10,IvPut10,IvMean10,...,IvMean360,IvMeanSkew360,IvCall720,IvPut720,IvMean720,IvMeanSkew720,IvCall1080,IvPut1080,IvMean1080,IvMeanSkew1080
4289,2002-03-21,0.3108,0.3058,0.3327,0.2425,0.2599,0.2861,0.3002,0.3063,0.3033,...,0.3466,0.0336,0.3641,0.3653,0.3647,0.0257,0.3641,0.3653,0.3647,0.0257
4290,2002-03-20,0.2944,0.3656,0.3297,0.2284,0.2719,0.2849,0.3081,0.3180,0.3130,...,0.3521,0.0301,0.3707,0.3670,0.3688,0.0241,0.3707,0.3670,0.3688,0.0241
4291,2002-03-19,0.2463,0.3252,0.3147,0.2281,0.2684,0.2825,0.2969,0.2982,0.2975,...,0.3468,0.0276,0.3722,0.3656,0.3689,0.0230,0.3722,0.3656,0.3689,0.0230
4292,2002-03-18,0.2835,0.3265,0.3267,0.2279,0.2774,0.2852,0.3144,0.3176,0.3160,...,0.3507,0.0306,0.3740,0.3643,0.3691,0.0240,0.3740,0.3643,0.3691,0.0240
4293,2002-03-15,0.2938,0.3288,0.3261,0.2335,0.2825,0.2843,0.3011,0.3159,0.3085,...,0.3499,0.0296,0.3729,0.3674,0.3702,0.0239,0.3729,0.3674,0.3702,0.0239


In [16]:
#new dataframe with only columns without NA
clean_vol =vol.dropna(axis = 'columns')
clean_vol.head()

,Date,Hv10,Hv20,Hv30,Phv10,Phv20,Phv30
0,2019-04-23,0.0951,0.0937,0.0938,0.1160,0.1065,0.1169
1,2019-04-22,0.0819,0.0825,0.1337,0.1080,0.1024,0.1217
2,2019-04-18,0.0839,0.0867,0.1528,0.1026,0.1004,0.1360
3,2019-04-17,0.0809,0.0786,0.1480,0.0843,0.0939,0.1355
4,2019-04-16,0.0691,0.0749,0.1461,0.0855,0.0952,0.1361


clean_vol now only contains colums without NaN.  Unfortunately the columns with IvMean, IvCall and IvPut have been lost.  I would like to retain those, as well as IvMeanSkew if possible, so it is important to find the NaN's in those columns and see how they can be dealth with.

In [17]:
vol.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4294 entries, 0 to 4293
Data columns (total 55 columns):
Date              4294 non-null datetime64[ns]
Hv10              4294 non-null float64
Hv20              4294 non-null float64
Hv30              4294 non-null float64
Phv10             4294 non-null float64
Phv20             4294 non-null float64
Phv30             4294 non-null float64
IvCall10          4291 non-null float64
IvPut10           4291 non-null float64
IvMean10          4291 non-null float64
IvMeanSkew10      4291 non-null float64
IvCall20          4291 non-null float64
IvPut20           4291 non-null float64
IvMean20          4291 non-null float64
IvMeanSkew20      4291 non-null float64
IvCall30          4291 non-null float64
IvPut30           4291 non-null float64
IvMean30          4291 non-null float64
IvMeanSkew30      4291 non-null float64
IvCall60          4291 non-null float64
IvPut60           4291 non-null float64
IvMean60          4291 non-null float64
IvMean

It seems like the remaining columns are missing only 3 values.  If these can be found we have many more columns to use as features.

In [18]:
na_vols = vol.isna()
na_vols.head()

,Date,Hv10,Hv20,Hv30,Phv10,Phv20,Phv30,IvCall10,IvPut10,IvMean10,...,IvMean360,IvMeanSkew360,IvCall720,IvPut720,IvMean720,IvMeanSkew720,IvCall1080,IvPut1080,IvMean1080,IvMeanSkew1080
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [19]:
vol[pd.isnull(vol).any(axis=1)]

,Date,Hv10,Hv20,Hv30,Phv10,Phv20,Phv30,IvCall10,IvPut10,IvMean10,...,IvMean360,IvMeanSkew360,IvCall720,IvPut720,IvMean720,IvMeanSkew720,IvCall1080,IvPut1080,IvMean1080,IvMeanSkew1080
3568,2005-02-16,0.0571,0.0780,0.0980,0.0852,0.1099,0.1197,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3679,2004-09-09,0.1487,0.1188,0.1442,0.1206,0.1232,0.1637,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3731,2004-06-25,0.1395,0.1389,0.1322,0.1813,0.1670,0.1511,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


I'm not sure why these 3 particular recoeds seem to be incomplete, but since my intention was to shorten the length of the time on the data set anyway I I will just start after 2005 and this should eliminate the issue.

In [20]:

clean_vol = pd.DataFrame(vol)
clean_vol = clean_vol.set_index('Date')
clean_vol.head(3)

,Hv10,Hv20,Hv30,Phv10,Phv20,Phv30,IvCall10,IvPut10,IvMean10,IvMeanSkew10,...,IvMean360,IvMeanSkew360,IvCall720,IvPut720,IvMean720,IvMeanSkew720,IvCall1080,IvPut1080,IvMean1080,IvMeanSkew1080
Date,,,,,,,,,,,,,,,,,,,,,
2019-04-23,0.0951,0.0937,0.0938,0.1160,0.1065,0.1169,0.2831,0.2816,0.2824,0.0467,...,0.2088,0.0164,0.2108,0.2563,0.2336,0.0214,0.2108,0.2563,0.2336,0.0214
2019-04-22,0.0819,0.0825,0.1337,0.1080,0.1024,0.1217,0.2724,0.2745,0.2734,0.0708,...,0.2141,0.0187,0.2029,0.2570,0.2300,0.0196,0.2029,0.2570,0.2300,0.0196
2019-04-18,0.0839,0.0867,0.1528,0.1026,0.1004,0.1360,0.2651,0.2663,0.2657,0.0859,...,0.2103,0.0216,0.2105,0.2394,0.2250,0.0030,0.2105,0.2394,0.2250,0.0030


In [21]:
clean_vol.tail()

,Hv10,Hv20,Hv30,Phv10,Phv20,Phv30,IvCall10,IvPut10,IvMean10,IvMeanSkew10,...,IvMean360,IvMeanSkew360,IvCall720,IvPut720,IvMean720,IvMeanSkew720,IvCall1080,IvPut1080,IvMean1080,IvMeanSkew1080
Date,,,,,,,,,,,,,,,,,,,,,
2002-03-21,0.3108,0.3058,0.3327,0.2425,0.2599,0.2861,0.3002,0.3063,0.3033,0.0676,...,0.3466,0.0336,0.3641,0.3653,0.3647,0.0257,0.3641,0.3653,0.3647,0.0257
2002-03-20,0.2944,0.3656,0.3297,0.2284,0.2719,0.2849,0.3081,0.3180,0.3130,0.0804,...,0.3521,0.0301,0.3707,0.3670,0.3688,0.0241,0.3707,0.3670,0.3688,0.0241
2002-03-19,0.2463,0.3252,0.3147,0.2281,0.2684,0.2825,0.2969,0.2982,0.2975,0.0677,...,0.3468,0.0276,0.3722,0.3656,0.3689,0.0230,0.3722,0.3656,0.3689,0.0230
2002-03-18,0.2835,0.3265,0.3267,0.2279,0.2774,0.2852,0.3144,0.3176,0.3160,0.0646,...,0.3507,0.0306,0.3740,0.3643,0.3691,0.0240,0.3740,0.3643,0.3691,0.0240
2002-03-15,0.2938,0.3288,0.3261,0.2335,0.2825,0.2843,0.3011,0.3159,0.3085,0.0535,...,0.3499,0.0296,0.3729,0.3674,0.3702,0.0239,0.3729,0.3674,0.3702,0.0239


In [22]:
clean_vol1 =clean_vol['December-31-2017':'Jan-01-2013']
clean_vol1.head()

,Hv10,Hv20,Hv30,Phv10,Phv20,Phv30,IvCall10,IvPut10,IvMean10,IvMeanSkew10,...,IvMean360,IvMeanSkew360,IvCall720,IvPut720,IvMean720,IvMeanSkew720,IvCall1080,IvPut1080,IvMean1080,IvMeanSkew1080
Date,,,,,,,,,,,,,,,,,,,,,
2017-12-29,0.0359,0.1366,0.2005,0.0924,0.1243,0.1632,0.1230,0.1185,0.1208,0.0553,...,0.2134,0.0096,0.1901,0.2472,0.2186,-0.0015,0.1899,0.2478,0.2188,-0.0024
2017-12-28,0.0503,0.1571,0.2115,0.1003,0.1289,0.1683,0.1244,0.1221,0.1232,0.0338,...,0.2094,0.0096,0.1927,0.2499,0.2213,0.0080,0.1926,0.2516,0.2221,0.0079
2017-12-27,0.0502,0.1571,0.2115,0.0990,0.1284,0.1680,0.1331,0.1283,0.1307,0.0833,...,0.2096,0.0082,0.1973,0.2510,0.2242,-0.0005,0.1974,0.2528,0.2251,-0.0011
2017-12-26,0.1726,0.1670,0.2161,0.1418,0.1392,0.1698,0.1426,0.1307,0.1366,0.0822,...,0.2076,0.0115,0.2010,0.2511,0.2260,0.0156,0.2013,0.2538,0.2276,0.0160
2017-12-22,0.1807,0.1661,0.2159,0.1513,0.1482,0.1701,0.1262,0.1102,0.1182,0.0376,...,0.2064,0.0140,0.1920,0.2493,0.2206,-0.0039,0.1912,0.2526,0.2219,-0.0059


In [23]:

prices = pd.DataFrame(MSFT)
prices = prices[['Close', 'Date']]
prices = prices.set_index('Date')
prices.head(3)


,Close
Date,
2018-03-27,89.47
2018-03-26,93.78
2018-03-23,87.18


In [24]:
prices1 = pd.DataFrame(MSFT)
#prices1 = prices1[['Close', 'Date','Adj. Close']]
prices1 = prices1.set_index('Date')
prices1= prices1['December-31-2017':'Jan-01-2013']
prices1.tail()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2013-01-08,26.75,26.79,26.4600,26.55,44703100.0,0.0,1.0,23.432378,23.467417,23.178345,23.257183,44703100.0
2013-01-07,26.77,26.88,26.6400,26.69,37110400.0,0.0,1.0,23.449898,23.546255,23.336021,23.379820,37110400.0
2013-01-04,27.27,27.34,26.7300,26.74,52521100.0,0.0,1.0,23.887886,23.949204,23.414859,23.423618,52521100.0
2013-01-03,27.63,27.65,27.1600,27.25,48294400.0,0.0,1.0,24.203238,24.220757,23.791529,23.870367,48294400.0
2013-01-02,27.25,27.73,27.1499,27.62,52899300.0,0.0,1.0,23.870367,24.290835,23.782681,24.194478,52899300.0


In [25]:
MSFTdata = prices1.join(clean_vol1)
MSFTdata.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,...,IvMean360,IvMeanSkew360,IvCall720,IvPut720,IvMean720,IvMeanSkew720,IvCall1080,IvPut1080,IvMean1080,IvMeanSkew1080
Date,,,,,,,,,,,,,,,,,,,,,
2017-12-29,85.63,86.0500,85.500,85.54,18162779.0,0.0,1.0,85.63,86.0500,85.500,...,0.2134,0.0096,0.1901,0.2472,0.2186,-0.0015,0.1899,0.2478,0.2188,-0.0024
2017-12-28,85.90,85.9300,85.550,85.72,9872795.0,0.0,1.0,85.90,85.9300,85.550,...,0.2094,0.0096,0.1927,0.2499,0.2213,0.0080,0.1926,0.2516,0.2221,0.0079
2017-12-27,85.65,85.9800,85.215,85.71,13000828.0,0.0,1.0,85.65,85.9800,85.215,...,0.2096,0.0082,0.1973,0.2510,0.2242,-0.0005,0.1974,0.2528,0.2251,-0.0011
2017-12-26,85.31,85.5346,85.030,85.40,9737412.0,0.0,1.0,85.31,85.5346,85.030,...,0.2076,0.0115,0.2010,0.2511,0.2260,0.0156,0.2013,0.2538,0.2276,0.0160
2017-12-22,85.40,85.6300,84.920,85.51,14033977.0,0.0,1.0,85.40,85.6300,84.920,...,0.2064,0.0140,0.1920,0.2493,0.2206,-0.0039,0.1912,0.2526,0.2219,-0.0059


In [26]:
MSFTdata.to_pickle('MSFTdata.pickle')

In [27]:
# plot interactive price chart of Microsoft Prices
prices.iplot()


In [28]:
splits = MSFT[MSFT['Split Ratio']!= 1.0]
print(splits)

           Date   Open   High    Low  Close      Volume  Ex-Dividend  \
3802 2003-02-18  24.62  24.99  24.40  24.96  28707750.0          0.0   
4779 1999-03-29  90.12  92.62  87.87  92.37  19944250.0          0.0   
5055 1998-02-23  80.94  81.69  79.37  81.62  15100450.0          0.0   
5358 1996-12-09  78.37  81.87  78.00  81.75   5919900.0          0.0   
6003 1994-05-23  49.00  51.00  48.50  50.56   2342100.0          0.0   
6493 1992-06-15  75.00  78.00  75.00  75.75   1144600.0          0.0   
6737 1991-06-27  67.75  68.25  66.75  68.00    784844.0          0.0   
7041 1990-04-16  61.00  61.75  60.13  60.75    274089.0          0.0   
7690 1987-09-21  53.50  57.25  53.00  53.50    297044.0          0.0   

      Split Ratio  Adj. Open  Adj. High   Adj. Low  Adj. Close  Adj. Volume  
3802          2.0  16.389318  16.635624  16.242866   16.615653   28707750.0  
4779          2.0  29.996046  30.828160  29.247144   30.744949   39888500.0  
5055          2.0  13.470262  13.595079  13.2

In [29]:
prices['Feb-20-2003': 'Feb-14-2003']

,Close
Date,
2003-02-20,24.14
2003-02-19,24.53
2003-02-18,24.96
2003-02-14,48.30


The split dates have now been determined.  Also by looking at the previous day it is clear that on whatever day the split occurs, the stock opens at that price in the morning after closing at the pre-split price the previous day.

Looking at the data I see there is actually an adjusted close column that should take into account the stock splits, so there is no need to do the math manually.  Going forward the adjusted close will be used as the default price for the stock.

The data has now been filtered and combined and is ready for both presentation and analysis.